# Assignment 2 – Lecture 2

## Lecture 2, Part 3

1. Start by creating a procedure to split the dataset into training and test sets. The proportion must be 80% for training and 20% for testing. Show your procedure working by plotting a figure with 3 subplots. The first plot must be the dataset with all data. The second must be the training set and the third the test set. 


In [ ]:
import csv
import numpy as np
import matplotlib.pyplot as plt

# Load and preprocess the dataset
data = np.loadtxt("./resources/datasets/secret_polynomial.csv", delimiter=',', skiprows=1)
X, y = data[:, 0], data[:, 1]

# Split the dataset into training and test sets (80% training, 20% testing)
np.random.seed(42)  # Ensure reproducibility
indices = np.random.permutation(len(X))
test_size = int(0.2 * len(X))

train_indices = indices[test_size:]
test_indices = indices[:test_size]

X_train, X_test = X[train_indices], X[test_indices]
y_train, y_test = y[train_indices], y[test_indices]

# Plotting the dataset
fig, axs = plt.subplots(1, 3, figsize=(15, 5))

# Define the limits for the plots based on the full dataset
x_limits = (min(X), max(X))
y_limits = (min(y), max(y))

# Plot all data
axs[0].scatter(X, y, color='blue', label='All Data')
axs[0].set_title('All Data')
axs[0].set_xlabel('X')
axs[0].set_ylabel('y')
axs[0].legend()
axs[0].set_xlim(x_limits)
axs[0].set_ylim(y_limits)

# Plot training data
axs[1].scatter(X_train, y_train, color='green', label='Training Set')
axs[1].set_title('Training Set')
axs[1].set_xlabel('X')
axs[1].set_ylabel('y')
axs[1].legend()
axs[1].set_xlim(x_limits)
axs[1].set_ylim(y_limits)

# Plot test data
axs[2].scatter(X_test, y_test, color='red', label='Test Set')
axs[2].set_title('Test Set')
axs[2].set_xlabel('X')
axs[2].set_ylabel('y')
axs[2].legend()
axs[2].set_xlim(x_limits)
axs[2].set_ylim(y_limits)

plt.tight_layout()
plt.show()

2. Now fit and plot (e.g., using subplots) all polynomial models for degrees $d\in [1,6]$. Observe your figure and decide which degree gives the best fit. Motivate your answer.

When we randomly split the data into 80% training and 20% testing sets, using a fixed seed of 42, the best results are observed at polynomial degree 6. For this specific combination of training and testing data, polynomial degree 6 yields the lowest Mean Squared Error (MSE) and the highest coefficient of determination (R²) for the testing data.

Here is the comparison table:

Degree                           |                  MSE                  |            Relative Change (%)         |                  R^2                
---------------------------------|---------------------------------------|----------------------------------------|--------------------------------------
1                                | 12,952                                | -                                      | 0.276
<span style="color:red">2</span> | <span style="color:red">13,235</span> | <span style="color:red">+2.2%</span>   | <span style="color:red">0.260</span>
3                                | 4,434                                 | -66.5%                                 | 0.752
4                                | 4,270                                 | -3.7%                                  | 0.761
5                                | 4,238                                 | -0.7%                                  | 0.763
<span style="color:lime">6</span>| <span style="color:lime">4,215</span> | <span style="color:lime">-0.5%</span>  | <span style="color:lime">0.764</span>

In [ ]:
import importlib
import MachineLearningModel

importlib.reload(MachineLearningModel)
from MachineLearningModel import RegressionModelNormalEquation

def plot_polynomial_models(degrees, X_train, y_train, X_test, y_test, X):
    fig, axs = plt.subplots(2, 3, figsize=(20, 15))
    axs = axs.ravel()

    for i, degree in enumerate(degrees):
        model = RegressionModelNormalEquation(degree=degree)
        X_train_normalized = model.normalize(X_train).reshape(-1, 1)
        X_test_normalized = model.normalize(X_test).reshape(-1, 1)
        
        model.fit(X_train_normalized, y_train)
        
        axs[i].scatter(X_train_normalized, y_train, color='green', label='Training Set')
        axs[i].scatter(X_test_normalized, y_test, color='red', label='Test Set')
        
        x_vals = np.linspace(min(model.normalize(X)), max(model.normalize(X)), 100).reshape(-1, 1)
        y_vals = model.predict(x_vals)
        
        axs[i].plot(x_vals, y_vals, color='blue', linewidth=2, label=f'Degree {degree}')
        axs[i].set_title(f'Polynomial Degree {degree}')
        axs[i].set_xlabel('X')
        axs[i].set_ylabel('y')
        axs[i].legend()

        r2 = model.score(X_test_normalized, y_test)
        mse_train = model.evaluate(X_train_normalized, y_train)
        mse_test = model.evaluate(X_test_normalized, y_test)
        axs[i].text(0.95, 0.05, f'MSE (train): {int(mse_train)}\nMSE (test): {int(mse_test)}\nR$^2$: {r2:.3f}', 
                    ha='right', va='bottom', transform=axs[i].transAxes, fontsize=12, bbox=dict(facecolor='white', alpha=0.5))
        
        # Print mse test and R^2 into the console for each polynomial degree
        print(f'Degree {degree}: MSE (test) = {int(mse_test)}, R^2 = {r2:.3f}')

    plt.tight_layout()
    plt.show()

degrees = range(1, 7)
plot_polynomial_models(degrees, X_train, y_train, X_test, y_test, X)


3. To increase the confidence of your answer, you must divide the data into training and test sets and make repeated runs with shuffled data (at least 20 runs). You must decide on the best way to make this decision. By using this approach, what is your decision and why?

First, we need to address two main challenges when defining the best polynomial model:

1. **Outliers**: Given the relatively small dataset, the 80% of training points we select might all be on one side of the graph, creating a dead zone for training.
2. **Variance**: Due to the small dataset size, results can vary significantly between different runs, leading to high variance in model performance.

To illustrate these issues, below I will show examples of the 3 worst and the 3 best shuffles, using seeds from 1 to 200.

In [ ]:
importlib.reload(MachineLearningModel)
from MachineLearningModel import RegressionModelNormalEquation

def evaluate_model(degree, X, y, test_size, seed):
    np.random.seed(seed)
    indices = np.random.permutation(X.shape[0])
    test_indices = indices[:test_size]
    train_indices = indices[test_size:]

    X_train, X_test = X[train_indices], X[test_indices]
    y_train, y_test = y[train_indices], y[test_indices]

    model = RegressionModelNormalEquation(degree=degree)
    X_train_normalized = model.normalize(X_train)
    X_test_normalized = model.normalize(X_test)

    model.fit(X_train_normalized, y_train)

    x_vals = np.linspace(min(model.normalize(X)), max(model.normalize(X)), 100)
    y_vals = model.predict(x_vals)

    r2 = model.score(X_test_normalized, y_test)
    return r2, x_vals, y_vals, X_test_normalized, y_test

def single_evaluation(degree, X, y, test_size=0.2):
    test_size = int(X.shape[0] * test_size)
    seeds = range(1, 201)
    seed_r2_pairs = [(seed, evaluate_model(degree, X, y, test_size, seed)[0]) for seed in seeds]

    lowest_r2_seeds = sorted(seed_r2_pairs, key=lambda x: x[1])[:3]
    highest_r2_seeds = sorted(seed_r2_pairs, key=lambda x: x[1], reverse=True)[:3]

    fig, axs = plt.subplots(2, 3, figsize=(20, 10))

    for i, (seed, r2) in enumerate(lowest_r2_seeds + highest_r2_seeds):
        r2, x_vals, y_vals, X_test_normalized, y_test = evaluate_model(degree, X, y, test_size, seed)
        row = 0 if i < 3 else 1
        col = i % 3
        color = 'red' if row == 0 else 'green'
        axs[row, col].plot(x_vals, y_vals, color=color, alpha=0.3, label=f'$R^2$: {r2:.3f}')
        axs[row, col].scatter(X_test_normalized, y_test, color='blue', label='Test Data Points')
        axs[row, col].set_title(f'Polynomial Degree {degree} (Seed: {seed})')
        axs[row, col].set_xlabel('X')
        axs[row, col].set_ylabel('y')
        axs[row, col].legend()

    plt.tight_layout()
    plt.show()

single_evaluation(4, X, y, test_size=0.2)

Depending on the shuffle, the R² value can drop as low as -4.

Run the cell below to generate a boxplot from 200 different runs, including outliers. The result is quite horrible due to the anomalies caused by a few dozen unlucky shuffles.

Interesting observation! If you run the cell multiple times, you might notice that polynomial degrees 4, 5, and 3 are the most affected by this issue, while degrees 1, 2, and 6 are the least affected.

In [ ]:
def repeated_evaluation(degrees, X, y, n_runs=20, test_size=0.2, showfliers=True, print_means=False):
    results = {degree: {'r2_scores': [], 'mse_test': []} for degree in degrees}
    n_samples = X.shape[0]
    test_size = int(n_samples * test_size)
    
    for _ in range(n_runs):
        np.random.seed(None) 
        indices = np.random.permutation(n_samples)
        
        test_indices = indices[:test_size]
        train_indices = indices[test_size:]
        
        X_train, X_test = X[train_indices], X[test_indices]
        y_train, y_test = y[train_indices], y[test_indices]
        
        for degree in degrees:
            model = RegressionModelNormalEquation(degree=degree)
            X_train_normalized = model.normalize(X_train)
            X_test_normalized = model.normalize(X_test).reshape(-1, 1)
            
            model.fit(X_train_normalized, y_train)
            
            r2 = model.score(X_test_normalized, y_test)
            mse_test = model.evaluate(X_test_normalized, y_test)
            
            results[degree]['r2_scores'].append(r2)
            results[degree]['mse_test'].append(mse_test)

    if print_means:
        mean_r2_scores = {degree: round(np.mean(results[degree]['r2_scores']), 3) for degree in degrees}
        print("Mean R^2 values for each polynomial degree:")
        print(f"{'Degree':<10}{'Mean R^2':<10}")
        print("-" * 20)
        for degree, mean_r2 in mean_r2_scores.items():
            print(f"{degree:<10}{mean_r2:<10}")

    fig, axs = plt.subplots(1, 2, figsize=(10, 5))

    for i, metric in enumerate(['r2_scores', 'mse_test']):
        data = [results[degree][metric] for degree in degrees]
        axs[i].boxplot(data, labels=[f'{d}' for d in degrees], showfliers=showfliers)
        axs[i].set_title(metric.replace('_', ' ').title())
        axs[i].set_xlabel('Polynomial Degree')
        axs[i].set_ylabel(metric.replace('_', ' ').title())

    plt.tight_layout()
    plt.show()

degrees = range(1, 7)
repeated_evaluation(degrees, X, y, n_runs=200, test_size=0.2)

Now, let's remove the outliers and rerun the function to observe the true impact of polynomial degree on the coefficient of determination.

In [ ]:
degrees = range(1, 7)
repeated_evaluation(degrees, X, y, n_runs=200, test_size=0.2, showfliers=False, print_means=True)

The chart clearly shows that polynomials of degree 1 and 2 do not efficiently regress the dataset, resulting in relatively low R^2 scores. In contrast, polynomials of degree 3 to 6 exhibit positive skewness. Based on our previous findings about variance and outliers, this skewness is likely due to the unlucky shuffles rather than the model itself. 

When selecting the polynomial degree that best addresses these dataset issues while providing precise results, degree 6 is still a reliable choice. Although polynomial degrees 3, 4 and 5 produce comparable outcomes, degree 6 generally offers a slightly higher mean and less extreme minimum. This can be easily verified by running the previous cell multiple times.